In [1]:
#Import libraries
import os
import numpy as np
import tensorflow as tf
from PIL import Image

In [2]:
#Function to extract labels from files
def extract_labels(file_path):
    with open(file_path) as file_labels:

        file_lines = file_labels.readlines()

        file_shape_labels= [None] * len(file_lines)
        file_position_labels = [None] * len(file_lines)
        file_phoneme_labels = [None] * len(file_lines)
        file_second_phoneme_label = [None] * len(file_lines)
        
        for i, line in enumerate(file_lines):
            file_line = line.split()

            file_shape_labels[i]  = int(file_line[1])
            file_position_labels[i] = int(file_line[2])
            file_phoneme_labels[i] = int(file_line[3])
            file_second_phoneme_label[i] = int(file_line[4])
            
    return np.array(file_shape_labels), np.array(file_position_labels), np.array(file_phoneme_labels), np.array(file_second_phoneme_label)

In [3]:
#Funtion to randomly shuffle data
def shuffle_data(images, shape_labels, position_labels, phoneme_labels, second_phoneme_labels):
    #index = np.arange(len(labels))
    #np.shuffle(index)
    index = np.random.choice(np.arange(len(phoneme_labels)), replace = False, size = len(phoneme_labels))
    
    shuffled_images = images[index]
    shuffled_shape_labels = shape_labels[index]
    shuffled_position_labels = position_labels[index]
    shuffled_phoneme_labels = phoneme_labels[index]
    shuffled_second_phoneme_labels = second_phoneme_labels[index]
    return shuffled_images, shuffled_shape_labels, shuffled_position_labels, shuffled_phoneme_labels, shuffled_second_phoneme_labels

In [4]:
#Function to split data
def split_data(data, split_ratio=0.9):
    partition = round(len(data) * split_ratio)
    
    train = data[:partition]
    test = data[partition:]
    return train, test

In [5]:
#Function to prepare dataset to input network
def create_dataset(image_dir):
    dataset = []
    for image_name in os.listdir(image_dir):
        image = Image.open(image_dir + image_name)
        image_as_array = np.asarray(image)
        dataset.append(image_as_array)
    return np.array(dataset)

In [6]:
#Function to prepare dataset as grey images to input network
def create_grey_dataset(image_dir):
    grey_dataset = []
    for image_name in os.listdir(image_dir):
        image = Image.open(image_dir + image_name)
        image_as_array = np.asarray(image)
        r, g, b = image_as_array[:,:,0], image_as_array[:,:,1], image_as_array[:,:,2]
        grey_image = (0.3*r) + (0.59*g) +(0.11*b)
        grey_dataset.append(grey_image)
    return np.array(grey_dataset)

In [7]:
#Function to concatenate outputs of two networks into single vector
def join_outputs(out1, out2):
    joint_output = []
    for i in range(len(out1)):
        joint = np.concatenate((out1[i],out2[i]))
        joint_output.append(joint)
    return np.array(joint_output)

In [8]:
#Open image folders
consonants_dir = os.path.join('/Users/User/Desktop/MSc Project/English Cued Speech/ConsonantImages/')
vowels_dir = os.path.join('/Users/User/Desktop/MSc Project/English Cued Speech/VowelImages/')
combined_dir = os.path.join('/Users/User/Desktop/MSc Project/English Cued Speech/PhonemeImages/')

#Open label files
consonant_labels = "/Users/User/Desktop/MSc Project/English Cued Speech/Labels_Consonant.txt"
vowel_labels = "/Users/User/Desktop/MSc Project/English Cued Speech/Labels_Vowel.txt"
combined_labels = "/Users/User/Desktop/MSc Project/English Cued Speech/Labels_All.txt"

In [9]:
#Extract labels
(vowel_shape_labels, vowel_position_labels, vowel_phoneme_labels, not_used) = extract_labels(vowel_labels)
(consonant_shape_labels, consonant_position_labels, consonant_phoneme_labels, or_this) = extract_labels(consonant_labels)
(combined_shape_labels, combined_position_labels, combined_consonant_labels, combined_vowel_labels) = extract_labels(combined_labels)

In [ ]:
#Vowels Model

In [10]:
#Prepare images of vowels to input into network
vowel_images = create_dataset(vowels_dir)
vowel_images = vowel_images.reshape(len(vowel_images), 256, 256, 3)

In [11]:
#Randomly shuffle vowel images and labels
shuffled_vowels, shuffled_vowel_shape_labels, shuffled_vowel_position_labels, shuffled_vowel_phoneme_labels, non_used = shuffle_data(vowel_images, vowel_shape_labels, vowel_position_labels, vowel_phoneme_labels, not_used)

#Split training and test data
vowel_train, vowel_test = split_data(shuffled_vowels)
vowel_shape_labels_train, vowel_shape_labels_test = split_data(shuffled_vowel_shape_labels)
vowel_position_labels_train, vowel_position_labels_test = split_data(shuffled_vowel_position_labels)
vowel_phoneme_labels_train, vowel_phoneme_labels_test = split_data(shuffled_vowel_phoneme_labels)

In [12]:
#Build the network to recognise hand position
hand_position_model = tf.keras.models.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 3)),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(5, activation='softmax')
])

#Compile the network
hand_position_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the network
hand_position_model.fit(vowel_train, vowel_position_labels_train, epochs=5)

Epoch 1/5
33/33 [==============================] - 34s 1s/step - loss: 0.8214 - accuracy: 0.7274
Epoch 2/5
33/33 [==============================] - 36s 1s/step - loss: 0.1983 - accuracy: 0.9408
Epoch 3/5
33/33 [==============================] - 36s 1s/step - loss: 0.1322 - accuracy: 0.9709
Epoch 4/5
33/33 [==============================] - 37s 1s/step - loss: 0.1150 - accuracy: 0.9728
Epoch 5/5
33/33 [==============================] - 37s 1s/step - loss: 0.0898 - accuracy: 0.9835


In [13]:
#Test the network and print it's performance
hand_position_test_loss, hand_position_test_accuracy = hand_position_model.evaluate(vowel_test, vowel_position_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(hand_position_test_loss, hand_position_test_accuracy*100))

4/4 [==============================] - 1s 295ms/step - loss: 0.1863 - accuracy: 0.9652
Test loss: 0.18631866574287415, Test accuracy: 96.52174115180969


In [14]:
#Build network to recognise vowel phoneme
vowel_phoneme_model = tf.keras.models.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 3)),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(13, activation='softmax')
])

#Complie the network
vowel_phoneme_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the network
vowel_phoneme_model.fit(vowel_train, vowel_phoneme_labels_train, epochs=5)

Epoch 1/5
33/33 [==============================] - 35s 1s/step - loss: 1.6215 - accuracy: 0.5092
Epoch 2/5
33/33 [==============================] - 35s 1s/step - loss: 1.0689 - accuracy: 0.6130
Epoch 3/5
33/33 [==============================] - 36s 1s/step - loss: 0.9317 - accuracy: 0.6440
Epoch 4/5
33/33 [==============================] - 36s 1s/step - loss: 0.7878 - accuracy: 0.6838
Epoch 5/5
33/33 [==============================] - 36s 1s/step - loss: 0.7738 - accuracy: 0.6925


In [15]:
#Test the network and print it's performance
vowel_phoneme_test_loss, vowel_phoneme_test_accuracy = vowel_phoneme_model.evaluate(vowel_test, vowel_phoneme_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(vowel_phoneme_test_loss, vowel_phoneme_test_accuracy*100))

4/4 [==============================] - 1s 314ms/step - loss: 0.8962 - accuracy: 0.6696
Test loss: 0.8962281346321106, Test accuracy: 66.9565200805664


In [16]:
#Obtain decisions from both networks and concatenate into a single decision vector
hand_position_decision = hand_position_model(vowel_train)

vowel_phoneme_decision = vowel_phoneme_model(vowel_train)

vowel_decision = join_outputs(hand_position_decision, vowel_phoneme_decision)

In [17]:
#Obtain test vectors to test network on
hand_position_decision_test = hand_position_model(vowel_test)

vowel_phoneme_decision_test = vowel_phoneme_model(vowel_test)

vowel_decision_test = join_outputs(hand_position_decision_test, vowel_phoneme_decision_test)

In [18]:
#Build network to make final decision of phoneme
vowel_decision_model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(16, activation='sigmoid'),
  tf.keras.layers.Dense(13, activation='softmax')
])

#Compile network
vowel_decision_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train network
vowel_decision_model.fit(vowel_decision, vowel_phoneme_labels_train, epochs=30)

Epoch 1/30
33/33 [==============================] - 0s 2ms/step - loss: 2.5144 - accuracy: 0.2211
Epoch 2/30
33/33 [==============================] - 0s 2ms/step - loss: 2.3510 - accuracy: 0.2173
Epoch 3/30
33/33 [==============================] - 0s 2ms/step - loss: 2.2358 - accuracy: 0.2173
Epoch 4/30
33/33 [==============================] - 0s 1ms/step - loss: 2.1508 - accuracy: 0.2173
Epoch 5/30
33/33 [==============================] - 0s 1ms/step - loss: 2.0855 - accuracy: 0.3298
Epoch 6/30
33/33 [==============================] - 0s 1ms/step - loss: 2.0310 - accuracy: 0.4462
Epoch 7/30
33/33 [==============================] - 0s 1ms/step - loss: 1.9806 - accuracy: 0.4462
Epoch 8/30
33/33 [==============================] - 0s 2ms/step - loss: 1.9331 - accuracy: 0.4462
Epoch 9/30
33/33 [==============================] - 0s 1ms/step - loss: 1.8837 - accuracy: 0.4462
Epoch 10/30
33/33 [==============================] - 0s 2ms/step - loss: 1.8341 - accuracy: 0.4462
Epoch 11/30
33/33 [

In [19]:
#Test network and print result
vowel_test_loss, vowel_test_accuracy = vowel_decision_model.evaluate(vowel_decision_test, vowel_phoneme_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(vowel_test_loss, vowel_test_accuracy*100))

4/4 [==============================] - 0s 3ms/step - loss: 1.1102 - accuracy: 0.6522
Test loss: 1.1102010011672974, Test accuracy: 65.21739363670349


In [ ]:
#Consonants Model

In [20]:
#Prepare images of consonants to input into network
consonant_images = create_dataset(consonants_dir)
consonant_images = consonant_images.reshape(len(consonant_images), 256, 256, 3)

In [21]:
#Randomly shuffle consonant images and labels
shuffled_consonants, shuffled_consonant_shape_labels, shuffled_consonant_position_labels, shuffled_consonant_phoneme_labels, nor_this = shuffle_data(consonant_images, consonant_shape_labels, consonant_position_labels, consonant_phoneme_labels, or_this)

#Split training and test data
consonant_train, consonant_test = split_data(shuffled_consonants)
consonant_shape_labels_train, consonant_shape_labels_test = split_data(shuffled_consonant_shape_labels)
consonant_position_labels_train, consonant_position_labels_test = split_data(shuffled_consonant_position_labels)
consonant_phoneme_labels_train, consonant_phoneme_labels_test = split_data(shuffled_consonant_phoneme_labels)

In [22]:
#Build the network to recognise hand shape
hand_shape_model = tf.keras.models.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 3)),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(9, activation='softmax')
])

#Compile the networ
hand_shape_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the network
hand_shape_model.fit(consonant_train, consonant_shape_labels_train, epochs=5)

Epoch 1/5
43/43 [==============================] - 47s 1s/step - loss: 2.0183 - accuracy: 0.2098
Epoch 2/5
43/43 [==============================] - 46s 1s/step - loss: 1.6329 - accuracy: 0.4114
Epoch 3/5
43/43 [==============================] - 33s 765ms/step - loss: 1.2034 - accuracy: 0.5930
Epoch 4/5
43/43 [==============================] - 53s 1s/step - loss: 0.8612 - accuracy: 0.7124
Epoch 5/5
43/43 [==============================] - 49s 1s/step - loss: 0.6225 - accuracy: 0.8058


In [23]:
#Test the network and print it's performance
hand_shape_test_loss, hand_shape_test_accuracy = hand_shape_model.evaluate(consonant_test, consonant_shape_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(hand_shape_test_loss, hand_shape_test_accuracy*100))

5/5 [==============================] - 2s 337ms/step - loss: 0.7584 - accuracy: 0.7000
Test loss: 0.7584484219551086, Test accuracy: 69.9999988079071


In [24]:
#Build network to recognise consonant phoneme
consonant_phoneme_model = tf.keras.models.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 3)),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(25, activation='softmax')
])

#Complie the network
consonant_phoneme_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the network
consonant_phoneme_model.fit(consonant_train, consonant_phoneme_labels_train, epochs=5)

Epoch 1/5
43/43 [==============================] - 46s 1s/step - loss: 2.9272 - accuracy: 0.1282
Epoch 2/5
43/43 [==============================] - 47s 1s/step - loss: 2.5543 - accuracy: 0.2609
Epoch 3/5
43/43 [==============================] - 46s 1s/step - loss: 2.1560 - accuracy: 0.3640
Epoch 4/5
43/43 [==============================] - 47s 1s/step - loss: 1.7239 - accuracy: 0.4841
Epoch 5/5
43/43 [==============================] - 45s 1s/step - loss: 1.3917 - accuracy: 0.5856


In [25]:
#Test the network and print it's performance
consonant_phoneme_test_loss, consonant_phoneme_test_accuracy = consonant_phoneme_model.evaluate(consonant_test, consonant_phoneme_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(consonant_phoneme_test_loss, consonant_phoneme_test_accuracy*100))

5/5 [==============================] - 2s 336ms/step - loss: 1.9401 - accuracy: 0.4200
Test loss: 1.9401021003723145, Test accuracy: 41.999998688697815


In [26]:
#Obtain decisions from both networks and concatenate into a single decision vector
hand_shape_decision = hand_shape_model(consonant_train)

consonant_phoneme_decision = consonant_phoneme_model(consonant_train)

consonant_decision = join_outputs(hand_shape_decision, consonant_phoneme_decision)

In [27]:
#Obtain test vectors to test network on
hand_shape_decision_test = hand_shape_model(consonant_test)

consonant_phoneme_decision_test = consonant_phoneme_model(consonant_test)

consonant_decision_test = join_outputs(hand_shape_decision_test, consonant_phoneme_decision_test)

In [28]:
#Build network to make final decision of phoneme
consonant_decision_model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(16, activation='sigmoid'),
  tf.keras.layers.Dense(25, activation='softmax')
])

#Compile network
consonant_decision_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train network
consonant_decision_model.fit(consonant_decision, consonant_phoneme_labels_train, epochs=30)

Epoch 1/30
43/43 [==============================] - 0s 2ms/step - loss: 3.1522 - accuracy: 0.0726
Epoch 2/30
43/43 [==============================] - 0s 2ms/step - loss: 3.0387 - accuracy: 0.0734
Epoch 3/30
43/43 [==============================] - 0s 2ms/step - loss: 2.9704 - accuracy: 0.1171
Epoch 4/30
43/43 [==============================] - 0s 2ms/step - loss: 2.9279 - accuracy: 0.1164
Epoch 5/30
43/43 [==============================] - 0s 2ms/step - loss: 2.8960 - accuracy: 0.1164
Epoch 6/30
43/43 [==============================] - 0s 2ms/step - loss: 2.8694 - accuracy: 0.1231
Epoch 7/30
43/43 [==============================] - 0s 2ms/step - loss: 2.8446 - accuracy: 0.1290
Epoch 8/30
43/43 [==============================] - 0s 2ms/step - loss: 2.8190 - accuracy: 0.1534
Epoch 9/30
43/43 [==============================] - 0s 2ms/step - loss: 2.7919 - accuracy: 0.1875
Epoch 10/30
43/43 [==============================] - 0s 2ms/step - loss: 2.7632 - accuracy: 0.1987: 0s - loss: 2.7482 

In [29]:
#Test network and print result
consonant_test_loss, consonant_test_accuracy = consonant_decision_model.evaluate(consonant_decision_test, consonant_phoneme_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(consonant_test_loss, consonant_test_accuracy*100))

5/5 [==============================] - 0s 3ms/step - loss: 2.0551 - accuracy: 0.3867
Test loss: 2.0550689697265625, Test accuracy: 38.66666555404663


In [ ]:
#Grey Vowels Model

In [30]:
#Convert images of consonants to greyscale and prepare to input into network
grey_vowel_images = create_grey_dataset(vowels_dir)
grey_vowel_images = grey_vowel_images.reshape(len(grey_vowel_images), 256, 256, 1)

In [31]:
#Randomly shuffle consonant images and labels
grey_shuffled_vowels, grey_shuffled_vowel_shape_labels, grey_shuffled_vowel_position_labels, grey_shuffled_vowel_phoneme_labels, grey_non_used = shuffle_data(grey_vowel_images, vowel_shape_labels, vowel_position_labels, vowel_phoneme_labels, not_used)

#Split training and test data
grey_vowel_train, grey_vowel_test = split_data(grey_shuffled_vowels)
grey_vowel_shape_labels_train, grey_vowel_shape_labels_test = split_data(grey_shuffled_vowel_shape_labels)
grey_vowel_position_labels_train, grey_vowel_position_labels_test = split_data(grey_shuffled_vowel_position_labels)
grey_vowel_phoneme_labels_train, grey_vowel_phoneme_labels_test = split_data(grey_shuffled_vowel_phoneme_labels)

In [33]:
#Build the network to recognise hand shape
grey_hand_position_model = tf.keras.models.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 1)),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(9, activation='softmax')
])

#Compile the networ
grey_hand_position_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the network
grey_hand_position_model.fit(grey_vowel_train, grey_vowel_position_labels_train, epochs=5)

Epoch 1/5
33/33 [==============================] - 30s 914ms/step - loss: 0.7084 - accuracy: 0.7633
Epoch 2/5
33/33 [==============================] - 32s 963ms/step - loss: 0.2104 - accuracy: 0.9399
Epoch 3/5
33/33 [==============================] - 30s 916ms/step - loss: 0.1458 - accuracy: 0.9622
Epoch 4/5
33/33 [==============================] - 30s 922ms/step - loss: 0.1116 - accuracy: 0.9748
Epoch 5/5
33/33 [==============================] - 30s 919ms/step - loss: 0.0854 - accuracy: 0.9835


In [34]:
#Test the network and print it's performance
grey_hand_position_test_loss, grey_hand_position_test_accuracy = grey_hand_position_model.evaluate(grey_vowel_test, grey_vowel_position_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(grey_hand_position_test_loss, grey_hand_position_test_accuracy*100))

4/4 [==============================] - 1s 251ms/step - loss: 0.1832 - accuracy: 0.9652
Test loss: 0.18320411443710327, Test accuracy: 96.52174115180969


In [35]:
#Build network to recognise consonant phoneme
grey_vowel_phoneme_model = tf.keras.models.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 1)),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(25, activation='softmax')
])

#Complie the network
grey_vowel_phoneme_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the network
grey_vowel_phoneme_model.fit(grey_vowel_train, grey_vowel_phoneme_labels_train, epochs=5)

Epoch 1/5
33/33 [==============================] - 31s 937ms/step - loss: 1.8695 - accuracy: 0.4423
Epoch 2/5
33/33 [==============================] - 31s 936ms/step - loss: 1.1885 - accuracy: 0.6120
Epoch 3/5
33/33 [==============================] - 31s 941ms/step - loss: 0.9571 - accuracy: 0.6547
Epoch 4/5
33/33 [==============================] - 31s 930ms/step - loss: 0.8026 - accuracy: 0.6916
Epoch 5/5
33/33 [==============================] - 30s 913ms/step - loss: 0.6962 - accuracy: 0.7284


In [36]:
#Test the network and print it's performance
grey_vowel_phoneme_test_loss, grey_vowel_phoneme_test_accuracy = grey_vowel_phoneme_model.evaluate(grey_vowel_test, grey_vowel_phoneme_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(grey_vowel_phoneme_test_loss, grey_vowel_phoneme_test_accuracy*100))

4/4 [==============================] - 1s 262ms/step - loss: 0.9839 - accuracy: 0.6174
Test loss: 0.9839266538619995, Test accuracy: 61.739128828048706


In [37]:
#Obtain decisions from both networks and concatenate into a single decision vector
grey_hand_position_decision = grey_hand_position_model(grey_vowel_train)

grey_vowel_phoneme_decision = grey_vowel_phoneme_model(grey_vowel_train)

grey_vowel_decision = join_outputs(grey_hand_position_decision, grey_vowel_phoneme_decision)

In [38]:
#Obtain test vectors to test network on
grey_hand_position_decision_test = grey_hand_position_model(grey_vowel_test)

grey_vowel_phoneme_decision_test = grey_vowel_phoneme_model(grey_vowel_test)

grey_vowel_decision_test = join_outputs(grey_hand_position_decision_test, grey_vowel_phoneme_decision_test)

In [39]:
#Build network to make final decision of phoneme
grey_vowel_decision_model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(16, activation='sigmoid'),
  tf.keras.layers.Dense(25, activation='softmax')
])

#Compile network
grey_vowel_decision_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train network
grey_vowel_decision_model.fit(grey_vowel_decision, grey_vowel_phoneme_labels_train, epochs=30)

Epoch 1/30
33/33 [==============================] - 0s 2ms/step - loss: 3.1564 - accuracy: 0.0582
Epoch 2/30
33/33 [==============================] - 0s 1ms/step - loss: 2.8559 - accuracy: 0.1707
Epoch 3/30
33/33 [==============================] - 0s 1ms/step - loss: 2.6193 - accuracy: 0.5131
Epoch 4/30
33/33 [==============================] - 0s 2ms/step - loss: 2.4413 - accuracy: 0.4539
Epoch 5/30
33/33 [==============================] - 0s 1ms/step - loss: 2.3060 - accuracy: 0.4539
Epoch 6/30
33/33 [==============================] - 0s 1ms/step - loss: 2.2027 - accuracy: 0.4549
Epoch 7/30
33/33 [==============================] - 0s 2ms/step - loss: 2.1242 - accuracy: 0.4549
Epoch 8/30
33/33 [==============================] - 0s 2ms/step - loss: 2.0584 - accuracy: 0.4568
Epoch 9/30
33/33 [==============================] - 0s 1ms/step - loss: 2.0002 - accuracy: 0.4568
Epoch 10/30
33/33 [==============================] - 0s 2ms/step - loss: 1.9446 - accuracy: 0.4568
Epoch 11/30
33/33 [

In [40]:
#Test network and print result
grey_vowel_test_loss, grey_vowel_test_accuracy = grey_vowel_decision_model.evaluate(grey_vowel_decision_test, grey_vowel_phoneme_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(grey_vowel_test_loss, grey_vowel_test_accuracy*100))

4/4 [==============================] - 0s 3ms/step - loss: 1.2972 - accuracy: 0.5739
Test loss: 1.2972033023834229, Test accuracy: 57.3913037776947


In [ ]:
#Grey Consonants Model

In [41]:
#Convert images of consonants to greyscale and prepare to input into network
grey_consonant_images = create_grey_dataset(consonants_dir)
grey_consonant_images = grey_consonant_images.reshape(len(grey_consonant_images), 256, 256, 1)

In [42]:
#Randomly shuffle consonant images and labels
grey_shuffled_consonants, grey_shuffled_consonant_shape_labels, grey_shuffled_consonant_position_labels, grey_shuffled_consonant_phoneme_labels, grey_nor_this = shuffle_data(grey_consonant_images, consonant_shape_labels, consonant_position_labels, consonant_phoneme_labels, or_this)

#Split training and test data
grey_consonant_train, grey_consonant_test = split_data(grey_shuffled_consonants)
grey_consonant_shape_labels_train, grey_consonant_shape_labels_test = split_data(grey_shuffled_consonant_shape_labels)
grey_consonant_position_labels_train, grey_consonant_position_labels_test = split_data(grey_shuffled_consonant_position_labels)
grey_consonant_phoneme_labels_train, grey_consonant_phoneme_labels_test = split_data(grey_shuffled_consonant_phoneme_labels)

In [43]:
#Build the network to recognise hand shape
grey_hand_shape_model = tf.keras.models.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 1)),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(9, activation='softmax')
])

#Compile the networ
grey_hand_shape_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the network
grey_hand_shape_model.fit(grey_consonant_train, grey_consonant_shape_labels_train, epochs=5)

Epoch 1/5
43/43 [==============================] - 40s 932ms/step - loss: 1.9734 - accuracy: 0.2906
Epoch 2/5
43/43 [==============================] - 40s 935ms/step - loss: 1.5897 - accuracy: 0.4566
Epoch 3/5
43/43 [==============================] - 41s 950ms/step - loss: 1.1460 - accuracy: 0.6182
Epoch 4/5
43/43 [==============================] - 40s 937ms/step - loss: 0.8670 - accuracy: 0.7094
Epoch 5/5
43/43 [==============================] - 40s 936ms/step - loss: 0.6674 - accuracy: 0.7695


In [44]:
#Test the network and print it's performance
grey_hand_shape_test_loss, grey_hand_shape_test_accuracy = grey_hand_shape_model.evaluate(grey_consonant_test, grey_consonant_shape_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(grey_hand_shape_test_loss, grey_hand_shape_test_accuracy*100))

5/5 [==============================] - 1s 267ms/step - loss: 0.9927 - accuracy: 0.7200
Test loss: 0.992749810218811, Test accuracy: 72.00000286102295


In [45]:
#Build network to recognise consonant phoneme
grey_consonant_phoneme_model = tf.keras.models.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 1)),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(25, activation='softmax')
])

#Complie the network
grey_consonant_phoneme_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the network
grey_consonant_phoneme_model.fit(grey_consonant_train, grey_consonant_phoneme_labels_train, epochs=5)

Epoch 1/5
43/43 [==============================] - 40s 922ms/step - loss: 3.0013 - accuracy: 0.1216
Epoch 2/5
43/43 [==============================] - 41s 958ms/step - loss: 2.7865 - accuracy: 0.1646
Epoch 3/5
43/43 [==============================] - 42s 973ms/step - loss: 2.5386 - accuracy: 0.2572
Epoch 4/5
43/43 [==============================] - 41s 942ms/step - loss: 2.2788 - accuracy: 0.3573
Epoch 5/5
43/43 [==============================] - 41s 949ms/step - loss: 1.9612 - accuracy: 0.4173


In [60]:
#Test the network and print it's performance
grey_consonant_phoneme_test_loss, grey_consonant_phoneme_test_accuracy = grey_consonant_phoneme_model.evaluate(grey_consonant_test, grey_consonant_phoneme_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(grey_consonant_phoneme_test_loss, grey_consonant_phoneme_test_accuracy*100))

5/5 [==============================] - 1s 280ms/step - loss: 2.2636 - accuracy: 0.4267
Test loss: 2.263645648956299, Test accuracy: 42.66666769981384


In [61]:
#Obtain decisions from both networks and concatenate into a single decision vector
grey_hand_shape_decision = grey_hand_shape_model(grey_consonant_train)

grey_consonant_phoneme_decision = grey_consonant_phoneme_model(grey_consonant_train)

grey_consonant_decision = join_outputs(grey_hand_shape_decision, grey_consonant_phoneme_decision)

In [62]:
#Obtain test vectors to test network on
grey_hand_shape_decision_test = grey_hand_shape_model(grey_consonant_test)

grey_consonant_phoneme_decision_test = grey_consonant_phoneme_model(grey_consonant_test)

grey_consonant_decision_test = join_outputs(grey_hand_shape_decision_test, grey_consonant_phoneme_decision_test)

In [63]:
#Build network to make final decision of phoneme
grey_consonant_decision_model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(16, activation='sigmoid'),
  tf.keras.layers.Dense(25, activation='softmax')
])

#Compile network
grey_consonant_decision_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train network
grey_consonant_decision_model.fit(grey_consonant_decision, grey_consonant_phoneme_labels_train, epochs=30)

Epoch 1/30
43/43 [==============================] - 0s 2ms/step - loss: 3.2502 - accuracy: 0.0645
Epoch 2/30
43/43 [==============================] - 0s 2ms/step - loss: 3.0733 - accuracy: 0.1023
Epoch 3/30
43/43 [==============================] - 0s 2ms/step - loss: 2.9590 - accuracy: 0.1023
Epoch 4/30
43/43 [==============================] - 0s 2ms/step - loss: 2.8756 - accuracy: 0.1023
Epoch 5/30
43/43 [==============================] - 0s 2ms/step - loss: 2.8073 - accuracy: 0.2350
Epoch 6/30
43/43 [==============================] - 0s 2ms/step - loss: 2.7460 - accuracy: 0.2224
Epoch 7/30
43/43 [==============================] - 0s 1ms/step - loss: 2.6860 - accuracy: 0.3225
Epoch 8/30
43/43 [==============================] - 0s 2ms/step - loss: 2.6245 - accuracy: 0.4173
Epoch 9/30
43/43 [==============================] - 0s 2ms/step - loss: 2.5609 - accuracy: 0.4233
Epoch 10/30
43/43 [==============================] - 0s 2ms/step - loss: 2.4925 - accuracy: 0.4240
Epoch 11/30
43/43 [

In [64]:
#Test network and print result
grey_consonant_test_loss, grey_consonant_test_accuracy = grey_consonant_decision_model.evaluate(grey_consonant_decision_test, grey_consonant_phoneme_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(grey_consonant_test_loss, grey_consonant_test_accuracy*100))

5/5 [==============================] - 0s 2ms/step - loss: 1.8380 - accuracy: 0.4800
Test loss: 1.8380109071731567, Test accuracy: 47.999998927116394
